# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 27 2023 1:40PM,256105,16,SSF_C2W5,Sartorius Stedim Filters Inc.,Released
1,Jan 27 2023 1:39PM,256106,19,MSP217977,"Methapharm, Inc.",Released
2,Jan 27 2023 1:20PM,256104,16,SSF_C1W5,Sartorius Stedim Filters Inc.,Released
3,Jan 27 2023 1:16PM,256103,10,0086335614,ISDIN Corporation,Released
4,Jan 27 2023 1:16PM,256103,10,0086335618,ISDIN Corporation,Released
5,Jan 27 2023 1:16PM,256103,10,0086335619,ISDIN Corporation,Released
6,Jan 27 2023 1:16PM,256103,10,0086335623,ISDIN Corporation,Released
7,Jan 27 2023 12:44PM,256100,10,PRF-48951.1,Bio-PRF,Released
8,Jan 27 2023 12:39PM,256097,10,0086335608,ISDIN Corporation,Released
9,Jan 27 2023 12:39PM,256097,10,0086335609,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,256100,Released,1
27,256103,Released,4
28,256104,Released,1
29,256105,Released,1
30,256106,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256100,NaN,NaN,1.0
256103,NaN,NaN,4.0
256104,NaN,NaN,1.0
256105,NaN,NaN,1.0
256106,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256028,0.0,0.0,1.0
256030,0.0,0.0,1.0
256046,26.0,12.0,1.0
256054,6.0,7.0,2.0
256060,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256028,0,0,1
256030,0,0,1
256046,26,12,1
256054,6,7,2
256060,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256028,0,0,1
1,256030,0,0,1
2,256046,26,12,1
3,256054,6,7,2
4,256060,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256028,,,1
1,256030,,,1
2,256046,26,12,1
3,256054,6,7,2
4,256060,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 27 2023 1:40PM,256105,16,Sartorius Stedim Filters Inc.
1,Jan 27 2023 1:39PM,256106,19,"Methapharm, Inc."
2,Jan 27 2023 1:20PM,256104,16,Sartorius Stedim Filters Inc.
3,Jan 27 2023 1:16PM,256103,10,ISDIN Corporation
7,Jan 27 2023 12:44PM,256100,10,Bio-PRF
8,Jan 27 2023 12:39PM,256097,10,ISDIN Corporation
37,Jan 27 2023 12:30PM,256096,10,ISDIN Corporation
41,Jan 27 2023 12:26PM,256095,10,Emerginnova
43,Jan 27 2023 12:26PM,256094,10,Emerginnova
44,Jan 27 2023 12:25PM,256092,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 27 2023 1:40PM,256105,16,Sartorius Stedim Filters Inc.,,,1
1,Jan 27 2023 1:39PM,256106,19,"Methapharm, Inc.",,,1
2,Jan 27 2023 1:20PM,256104,16,Sartorius Stedim Filters Inc.,,,1
3,Jan 27 2023 1:16PM,256103,10,ISDIN Corporation,,,4
4,Jan 27 2023 12:44PM,256100,10,Bio-PRF,,,1
5,Jan 27 2023 12:39PM,256097,10,ISDIN Corporation,,12,17
6,Jan 27 2023 12:30PM,256096,10,ISDIN Corporation,,,4
7,Jan 27 2023 12:26PM,256095,10,Emerginnova,,,2
8,Jan 27 2023 12:26PM,256094,10,Emerginnova,,,1
9,Jan 27 2023 12:25PM,256092,10,Emerginnova,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 27 2023 1:40PM,256105,16,Sartorius Stedim Filters Inc.,1,,
1,Jan 27 2023 1:39PM,256106,19,"Methapharm, Inc.",1,,
2,Jan 27 2023 1:20PM,256104,16,Sartorius Stedim Filters Inc.,1,,
3,Jan 27 2023 1:16PM,256103,10,ISDIN Corporation,4,,
4,Jan 27 2023 12:44PM,256100,10,Bio-PRF,1,,
5,Jan 27 2023 12:39PM,256097,10,ISDIN Corporation,17,12,
6,Jan 27 2023 12:30PM,256096,10,ISDIN Corporation,4,,
7,Jan 27 2023 12:26PM,256095,10,Emerginnova,2,,
8,Jan 27 2023 12:26PM,256094,10,Emerginnova,1,,
9,Jan 27 2023 12:25PM,256092,10,Emerginnova,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 27 2023 1:40PM,256105,16,Sartorius Stedim Filters Inc.,1,,
1,Jan 27 2023 1:39PM,256106,19,"Methapharm, Inc.",1,,
2,Jan 27 2023 1:20PM,256104,16,Sartorius Stedim Filters Inc.,1,,
3,Jan 27 2023 1:16PM,256103,10,ISDIN Corporation,4,,
4,Jan 27 2023 12:44PM,256100,10,Bio-PRF,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 27 2023 1:40PM,256105,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN
1,Jan 27 2023 1:39PM,256106,19,"Methapharm, Inc.",1.0,NaN,NaN
2,Jan 27 2023 1:20PM,256104,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN
3,Jan 27 2023 1:16PM,256103,10,ISDIN Corporation,4.0,NaN,NaN
4,Jan 27 2023 12:44PM,256100,10,Bio-PRF,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 27 2023 1:40PM,256105,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0
1,Jan 27 2023 1:39PM,256106,19,"Methapharm, Inc.",1.0,0.0,0.0
2,Jan 27 2023 1:20PM,256104,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0
3,Jan 27 2023 1:16PM,256103,10,ISDIN Corporation,4.0,0.0,0.0
4,Jan 27 2023 12:44PM,256100,10,Bio-PRF,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3841262,69.0,20.0,10.0
16,1024267,4.0,0.0,0.0
19,512152,2.0,12.0,26.0
21,768247,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3841262,69.0,20.0,10.0
1,16,1024267,4.0,0.0,0.0
2,19,512152,2.0,12.0,26.0
3,21,768247,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,69.0,20.0,10.0
1,16,4.0,0.0,0.0
2,19,2.0,12.0,26.0
3,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,69.0
1,16,Released,4.0
2,19,Released,2.0
3,21,Released,3.0
4,10,Executing,20.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,21
Status,,,,
Completed,10.0,0.0,26.0,0.0
Executing,20.0,0.0,12.0,0.0
Released,69.0,4.0,2.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,21
0,Completed,10.0,0.0,26.0,0.0
1,Executing,20.0,0.0,12.0,0.0
2,Released,69.0,4.0,2.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,21
0,Completed,10.0,0.0,26.0,0.0
1,Executing,20.0,0.0,12.0,0.0
2,Released,69.0,4.0,2.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()